# Rossmann Store Sales

Forecast sales using store, promotion, and competitor data

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

# 0.0 Motivation

Store managers attended a monthly status meeting.
CFO requested this solution during a monthly status meeting

# 0.1 The Root Cause of the Problem

CFO wants to renovate stores, so he needs a forecast of the next 6 weeks to renovate stores.

# 0.2 Problem Owner

CFO

# 0.3 Solution Format

- Granularity: Forecast sales per day / store in the next 6 weeks  
- Type of problem: Demand / sales forecast
- Potential methods: Time Series
- Delivery Format: Predictions accessed via cell phone.

# 1.0 Imports

In [ ]:
#1.1
from IPython.display       import Image
from IPython.core.display  import HTML
#1.3
import pandas as pd
#2.7.1
import numpy as np
import seaborn as sns

#2.1
import inflection

#2.5
import math

#1.0
import warnings
warnings.filterwarnings( 'ignore' )

## 1.1 Helper Function

In [ ]:
def cross_validation( x_training, kfold, model_name, model, verbose=False ):
    mae_list = []
    mape_list = []
    rmse_list = []
    for k in reversed( range( 1, kfold+1 ) ):
        if verbose:
            print( '\nKFold Number: {}'.format( k ) )
        # start and end date for validation 
        validation_start_date = x_training['date'].max() - datetime.timedelta( days=k*6*7)
        validation_end_date = x_training['date'].max() - datetime.timedelta( days=(k-1)*6*7)

        # filtering dataset
        training = x_training[x_training['date'] < validation_start_date]
        validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]

        # training and validation dataset
        # training
        xtraining = training.drop( ['date', 'sales'], axis=1 ) 
        ytraining = training['sales']

        # validation
        xvalidation = validation.drop( ['date', 'sales'], axis=1 )
        yvalidation = validation['sales']

        # model
        m = model.fit( xtraining, ytraining )

        # prediction
        yhat = m.predict( xvalidation )

        # performance
        m_result = ml_error( model_name, np.expm1( yvalidation ), np.expm1( yhat ) )

        # store performance of each kfold iteration
        mae_list.append(  m_result['MAE'] )
        mape_list.append( m_result['MAPE'] )
        rmse_list.append( m_result['RMSE'] )

    return pd.DataFrame( {'Model Name': model_name,
                          'MAE CV': np.round( np.mean( mae_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mae_list ), 2 ).astype( str ),
                          'MAPE CV': np.round( np.mean( mape_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mape_list ), 2 ).astype( str ),
                          'RMSE CV': np.round( np.mean( rmse_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( rmse_list ), 2 ).astype( str ) }, index=[0] )


def mean_percentage_error( y, yhat ):
    return np.mean( ( y - yhat ) / y )
     
    
def mean_absolute_percentage_error( y, yhat ):
    return np.mean( np.abs( ( y - yhat ) / y ) )

    
def ml_error( model_name, y, yhat ):
    mae = mean_absolute_error( y, yhat )
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    
    return pd.DataFrame( { 'Model Name': model_name, 
                           'MAE': mae, 
                           'MAPE': mape,
                           'RMSE': rmse }, index=[0] )

def cramer_v( x, y ):
    cm = pd.crosstab( x, y ).as_matrix()
    n = cm.sum()
    r, k = cm.shape
    
    chi2 = ss.chi2_contingency( cm )[0]
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
    
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1)
    
    return np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) )



def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [ ]:
jupyter_settings()

## 1.2 Data Dictionary

Most of the fields are self-explanatory. The following are descriptions for those that aren't.

Id - an Id that represents a (Store, Date) duple within the test set  
Store - a unique Id for each store  
Sales - the turnover for any given day (this is what you are predicting)  
Customers - the number of customers on a given day  
Open - an indicator for whether the store was open: 0 = closed, 1 = open  
StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None  
SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools  
StoreType - differentiates between 4 different store models: a, b, c, d  
Assortment - describes an assortment level: a = basic, b = extra, c = extended  
CompetitionDistance - distance in meters to the nearest competitor store  
CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened  
Promo - indicates whether a store is running a promo on that day  
Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating  
Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2  
PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store  

## 1.3 Loading Data

In [ ]:
#read the entire dataset in one go to memory
df_sales_raw = pd.read_csv('data/train.csv', low_memory = False)
df_store_raw = pd.read_csv('data/store.csv', low_memory = False)

#merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how = 'left', on = 'Store')

# 2.0 Data Description

In [ ]:
df2 = df_raw.copy()

## 2.1 Rename Columns

In [ ]:
cols_old = list( df2.columns )

snakecase = lambda x: inflection.underscore( x ) 

cols_new = list( map( snakecase, cols_old ) )

#rename columns
df2.columns = cols_new

## 2.2 Data Dimensions

In [ ]:
print( 'Number of rows: {}'.format( df2.shape[0] ) )
print( 'Number of columns: {}'.format( df2.shape[1] ) )

## 2.3 Checking Data Types

In [ ]:
# convert object to Datetime
df2[ 'date' ] = pd.to_datetime( df2[ 'date' ] )

In [ ]:
df2.dtypes

## 2.4 Check NA

In [ ]:
#number of missing values per column
df2.isna().sum()

In [ ]:
#colocar as 3 formas de resolver NA

In [ ]:
#percentage of missing values
( df2.isna().sum() / df2.shape[0] ).sort_values(ascending = False )

## 2.5 Fillout NA

In [ ]:
df2.sample(10)

In [ ]:
#promo_interval    
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
df2['promo_interval'].fillna(0, inplace = True)
df2['month_map'] = df2['date'].dt.month.map( month_map )
df2['is_promo'] = df2[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis = 1)

#promo2_since_week   
df2[ 'promo2_since_week' ] = df2.apply( lambda x: x[ 'date' ].week if math.isnan( x[ 'promo2_since_week' ] ) else x[ 'promo2_since_week' ],  axis = 1 )

#promo2_since_year     
df2[ 'promo2_since_year' ] = df2.apply( lambda x: x[ 'date' ].year if math.isnan( x[ 'promo2_since_year' ] ) else x[ 'promo2_since_year' ],  axis = 1 )

#competition_open_since_year 
df2[ 'competition_open_since_year' ] = df2.apply( lambda x: x[ 'date' ].year if math.isnan( x[ 'competition_open_since_year' ] ) else x[ 'competition_open_since_year' ],  axis = 1 )

#competition_open_since_month: Store has no nearest competitor or Store has the next competitor but does not know when it opened
df2[ 'competition_open_since_month' ] = df2.apply( lambda x: x[ 'date' ].month if math.isnan( x[ 'competition_open_since_month' ] ) else x[ 'competition_open_since_month' ],  axis = 1 )

#competition_distance: as it is empty, it must be assumed that there is no competitor nearby or he is too far away
df2[ 'competition_distance' ] = df2[ 'competition_distance' ].apply( lambda x:  200000  if math.isnan( x ) else x )


In [ ]:
df2.isnull().values.any()

In [ ]:
df2.isnull().sum().sum()

## 2.6 Change Data Types

In [ ]:
df2[ 'competition_open_since_month' ] = df2[ 'competition_open_since_month' ].astype( 'int32' )
df2[ 'competition_open_since_year' ] = df2[ 'competition_open_since_year' ].astype( 'int32' )
df2[ 'promo2_since_week' ] = df2[ 'promo2_since_week' ].astype( 'int32' )
df2[ 'promo2_since_year' ] = df2[ 'promo2_since_year' ].astype( 'int32' )

# convert object to Datetime
df2[ 'date' ] = pd.to_datetime( df2[ 'date' ] )

# in order to optimize memory, the allocation space must be reduced
df2 = df2.astype( { col: 'int32' for col in df2.select_dtypes( 'int64' ).columns } )
df2 = df2.astype( { col: 'float32' for col in df2.select_dtypes( 'float64' ).columns } )

In [ ]:
df2.dtypes

## 2.7 Descriptive Statistical

In [ ]:
df2.info()

In [ ]:
num_atributes = df2.select_dtypes(include = ['int32', 'float32'])
cat_atributes = df2.select_dtypes(exclude = ['int32', 'float32', 'datetime64[ns]'])

### 2.7.1 Numerical Attributes

In [ ]:
# Central tendency - median, mean
ct1 = pd.DataFrame(num_atributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_atributes.apply(np.median)).T
# Dispersion - std, min, max, range, skew, curtosis
d1 = pd.DataFrame( num_atributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_atributes.apply( min ) ).T
d3 = pd.DataFrame( num_atributes.apply( max ) ).T
d4 = pd.DataFrame( num_atributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_atributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_atributes.apply( lambda x: x.kurtosis() ) ).T

d7 = pd.DataFrame(num_atributes.quantile(.25)).T
d8 = pd.DataFrame(num_atributes.quantile(.50)).T
d9 = pd.DataFrame(num_atributes.quantile(.75)).T


# concatenate
m = pd.concat( [d2, d7, d8, d9, d3, d4, ct1, ct2, d1, d5, d6 ] ).T.reset_index()
m.columns = ['Atributes','Min','25%','50%','75%', 'Max','Range','Mean','Median','Std', 'Skew', 'Kurtosis']
m

#sales = media e mediana proximas (skew proxima de 0 logo eh proxima de normal, kurtosis = 1 ) 

In [ ]:
sns.distplot(df2['sales'])

In [ ]:
sns.distplot(df2['competition_distance'])

In [ ]:
#using Pandas Describe
num_atributes.describe().T

### 2.7.2 Categorical Attributes

In [ ]:
cat_atributes.apply(lambda x: x.unique().shape[0])

In [ ]:
cat_atributes.nunique().sort_values(ascending = False)

In [ ]:
cat_atributes.describe(include=[object])

In [ ]:
aux = df2[(df2['state_holiday'] != '0') & (df2['sales'] > 0)]

plt.subplot( 1, 3, 1 )
sns.boxplot( x='state_holiday', y='sales', data=aux )

plt.subplot( 1, 3, 2 )
sns.boxplot( x='store_type', y='sales', data=aux )

plt.subplot( 1, 3, 3 )
sns.boxplot( x='assortment', y='sales', data=aux )

### 2.8 Save Dataset Clean - Checkpoint

In [ ]:
df2.to_csv('data/02_Rossmann_Clean.csv', index = False)

# 3.0. Feature Engineering

In [ ]:
df3 = df2.copy()

## 3.1. Hypothesis Mind Map

In [ ]:
Image ('img/MindMapHypothesis.png') 

## 3.2. Hypothesis List


**1.** Stores with a larger assortment should sell more.

**2.** Stores with closer competitors should sell less.

**3.** Stores with longer competitors should sell more.

**4.** Stores with active promotions for longer should sell more.

**5.** Stores with more promotion days should sell more.

**6.** Stores with more consecutive promotions should sell more.

**7.** Stores open during the Christmas holiday should sell more.

**8.** Stores should sell more over the years.

**9.** Stores should sell more in the second half of the year.

**10.** Stores should sell more after the 10th of each month.

**11.** Stores should sell less on weekends.

**12.** Stores should sell less during school holidays.

## 3.3. Feature Engineering

In [ ]:
df3.columns

In [ ]:
#year
df3[ 'year' ] = df3[ 'date' ].dt.year

#month
df3[ 'month' ] = df3[ 'date' ].dt.month

#day
df3[ 'day' ] = df3[ 'date' ].dt.day

#weak of year
df3[ 'week_of_year' ] = df3[ 'date' ].dt.weekofyear

#year week
df3[ 'year_week' ] = df3[ 'date' ].dt.strftime( '%Y-%W')

#competition since
df3[ 'competition_since' ] = df3.apply(lambda x: datetime.datetime(year = x[ 'competition_open_since_year' ] , month = x[ 'competition_open_since_month' ], day = 1), axis = 1)
df3['competition_time_month'] = ( ( df3[ 'date' ] - df3[ 'competition_since' ] ) / 30).apply( lambda x: x.days ).astype( 'int32' )

#promo since
df3['promo_since'] = df3['promo2_since_year'].astype( str ) + '-' + df3['promo2_since_week'].astype( str )
df3['promo_since'] = df3['promo_since'].apply( lambda x:  datetime.datetime.strptime( x + '-1', '%Y-%W-%w') - datetime.timedelta( days = 7 ) )
df3['promo_time_week'] = ( ( df3[ 'date' ] - df3[ 'promo_since' ] ) / 7).apply( lambda x: x.days ).astype( 'int32' )

#assortment
df3['assortment'] = df3['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended')

#state_holiday
df3['state_holiday'] = df3['state_holiday'].apply(lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas'  if x == 'c' else 'regular_day')



In [ ]:
df3.head().T

## 3.4. Save Dataset - Checkpoint

In [ ]:
df2.to_csv('data/03_Rossmann.csv', index = False)

# 4.0. Filtering Variable

In [ ]:
df4 = df3.copy()

## 4.1 Line Filtering

In [ ]:
df4.head()

In [ ]:
df4 = df4[(df4['open'] != 0) & (df4['sales'] > 0)]


## 4.2 Column Selection

In [ ]:
df4 = df4.drop(['customers', 'open', 'promo_interval', 'month_map'], axis = 1)

## 4.3 Save Dataset - Checkpoint

In [ ]:
pd.to_csv('data/04_Rossmann.csv', index = False)

# 5.0 Exploratory Data Analysis

In [ ]:
df5 = df4.copy()